In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd

bunch = fetch_openml(data_id = 40975)
frame = pd.get_dummies(bunch.frame)
train, test = train_test_split(frame, stratify = bunch.frame['class'], random_state = 0)
print (len(frame), len(train), len(test))

1728 1296 432


In [4]:
import numpy as np
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules


frequent_itemsets = fpgrowth(train, use_colnames=True, min_support = 0.1)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(lug_boot_med, safety_low)",(class_unacc),0.114969,0.699846,0.114969,1.000000,1.428886,0.034508,inf
1,"(safety_low, persons_more)",(class_unacc),0.115741,0.699846,0.115741,1.000000,1.428886,0.034740,inf
2,"(safety_high, persons_2)",(class_unacc),0.101852,0.699846,0.101852,1.000000,1.428886,0.030571,inf
3,(maint_high),(class_unacc),0.255401,0.699846,0.189043,0.740181,1.057635,0.010302,1.155245
4,(safety_low),(class_unacc),0.344136,0.699846,0.344136,1.000000,1.428886,0.103294,inf
5,(persons_2),(class_unacc),0.332562,0.699846,0.332562,1.000000,1.428886,0.099820,inf
6,"(persons_2, safety_low)",(class_unacc),0.116512,0.699846,0.116512,1.000000,1.428886,0.034972,inf
7,"(persons_2, lug_boot_med)",(class_unacc),0.110340,0.699846,0.110340,1.000000,1.428886,0.033119,inf
8,"(persons_2, safety_med)",(class_unacc),0.114198,0.699846,0.114198,1.000000,1.428886,0.034277,inf
9,"(lug_boot_big, persons_2)",(class_unacc),0.111111,0.699846,0.111111,1.000000,1.428886,0.033350,inf


In [5]:
import numpy as np

confidences = []
for i, row in rules.iterrows():
    acondition = None
    for name in row.antecedents:
        if acondition is None:
            acondition = (test[name] == 1)
        else:
            acondition = acondition & (test[name] == 1)
    
    ccondition = None
    for name in row.consequents:
        if ccondition is None:
            ccondition = (test[name] == 1)
        else:
            ccondition = ccondition & (test[name] == 1)
    
    confidences.append (len(test[acondition & ccondition]) / len(test[acondition]))

print ("test condidence = %.2f, alen = %.2f, clen = %.2f" % (np.mean(confidences), 
                                                            rules.antecedents.apply(len).mean(), 
                                                            rules.consequents.apply(len).mean()))

test condidence = 0.93, alen = 1.58, clen = 1.00
